In [1]:



import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

import GridWorld_env
import gymnasium as gym
import random

In [18]:

# shared environment of [E1, E2, ... En] where Ei belongs to agent i
class sharedEnvironment:
    def __init__(self, state_dim, num_agents):
        self.state_dim = state_dim
        self.num_agents = num_agents
        # initialize the shared enviroinment = [E1, E2, ... En]
        self.env = [gym.make('GridWorld_env/GridWorld', dimension_size=state_dim) for i in range(num_agents)]

        for i in range(num_agents):
            self.env[i].reset()
        
    
    def getEnv(self, agentID):
        # todo, return shared environement 
        if (agentID < 0 or agentID >= self.num_agents):
            raise ValueError("Invalid agent ID")
        return self.env[agentID]

    def getOtherAgentsPos(self, agentID):
        # return the xor of all the other agents position except agentID
        otherAgentsPos = np.zeros((self.state_dim, self.state_dim, self.state_dim), dtype=int)

        for i in range(self.num_agents):
            if i != agentID:
                otherAgentsPos[self.env[i].agent_pos[0], self.env[i].agent_pos[1], self.env[i].agent_pos[2]] = 1
        toTensor = torch.tensor(otherAgentsPos).float()
        return toTensor 

    

    


class ActorCritic(nn.Module):
    # input dim (N, N, N, 4)  = (building zone, target, agentpos, other agent pos)
    def __init__(self, state_dim, action_dim):
        # state_dim^3 is the state space
        super(ActorCritic, self).__init__()

        self.state_dim = state_dim    
        self.action_dim = action_dim


        # just trying to replicate the paper
        self.leftConv = nn.Sequential(
            # todo: 1st layer conv should cut down the state dim in half. eg (10, 10, 10, 3) -> (5, 5, 5, 64)
            nn.Conv3d(in_channels=4, out_channels=64, kernel_size=3, stride=2, padding=1)  ,
            nn.Conv3d(in_channels = 64, out_channels = 128, kernel_size = 3, stride = 1, padding = 1),
            nn.Conv3d(in_channels = 128, out_channels = 256, kernel_size = 3, stride = 2, padding = 1)
        )

        self.rightConv = nn.Sequential(

        )

    def forward(self, state):
        x = self.leftConv(state);        
        return x 


sharedE = sharedEnvironment(state_dim=4, num_agents=2)
testA3c = ActorCritic(state_dim=4, action_dim=7)

env1 = sharedE.getEnv(0)
state, _, _, _, _ = env1.unwrapped.step(0)
state = torch.tensor(state).float()
otherAgentsPost = sharedE.getOtherAgentsPos(0)
# make otherAgentsPost [4, 4, 4]-> [1, 4, 4, 4]
otherAgentsPost = otherAgentsPost.unsqueeze(0)
fourChannelState = torch.cat((state, otherAgentsPost), dim=0)
print(fourChannelState.shape, " is shape of 4 chanelstate")


forwardOut = testA3c(fourChannelState)
print(forwardOut.shape, " is shape of forwardOut")


torch.Size([4, 4, 4, 4])  is shape of 4 chanelstate
torch.Size([256, 2, 2, 2])  is shape of forwardOut


In [ ]:
# thread function
def agentTrain(index: int , globalNet, optimizer):
    

    pass
    

In [ ]:
def globalTrain(numAgents):
    # master thread

    
    pass